In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.utils.data as data

For replication purposes use the following seed

In [2]:
torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Using GPU acceleration can greatly speed up computations in deep learning, as GPUs are designed to perform many parallel computations simultaneously. The torch.device function allows us to specify whether we want to use the GPU or the CPU for computations, and the code will automatically use the specified device for all operations.

In [3]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

3.3%

100.0%
100.0%


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100.0%
100.0%


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw


Data normalization is important in training neural networks because it helps to ensure that the input features are on a similar scale, which can improve the convergence of the optimization algorithm. The values 0.1307 and 0.3081 are the mean and standard deviation of the MNIST dataset. We use these values to normalize the input data so that the mean of the input data is 0 and the standard deviation is 1. 

In [4]:
train_loader = data.DataLoader(train_data, batch_size=64, shuffle=True, num_workers=2)
test_loader = data.DataLoader(test_data, batch_size=64, shuffle=False, num_workers=2)

In [5]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x

Input batch of MNIST images has a shape of (64, 1, 28, 28). With 64 being the batch size, 1 being the number of channels (grayscale), and 28x28 being the image dimensions. After the input is reshaped using x.view(-1, 784), the shape of the data is (64, 784). 
This is because the input data is flattened into a 1D tensor of length 784, which is then passed through the fully connected layers. The output of the final fully connected layer has a shape of (64, 10), which represents the predicted class scores for each image in the batch. The predicted class scores are then used to calculate the loss and perform backpropagation during training.

In [6]:
model = MLP().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [7]:
%%time
for epoch in range(20):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print('Epoch {} training loss: {:.3f}'.format(epoch+1, running_loss/len(train_loader)))

Epoch 1 training loss: 1.805
Epoch 2 training loss: 0.481
Epoch 3 training loss: 0.321
Epoch 4 training loss: 0.253
Epoch 5 training loss: 0.206
Epoch 6 training loss: 0.171
Epoch 7 training loss: 0.147
Epoch 8 training loss: 0.128
Epoch 9 training loss: 0.114
Epoch 10 training loss: 0.101
Epoch 11 training loss: 0.091
Epoch 12 training loss: 0.083
Epoch 13 training loss: 0.075
Epoch 14 training loss: 0.069
Epoch 15 training loss: 0.062
Epoch 16 training loss: 0.057
Epoch 17 training loss: 0.053
Epoch 18 training loss: 0.048
Epoch 19 training loss: 0.045
Epoch 20 training loss: 0.041
CPU times: user 29.5 s, sys: 3min 6s, total: 3min 36s
Wall time: 43.1 s


Backpropagation is the process of calculating the gradients of the loss function with respect to the model's parameters, which is used to update the model's parameters during optimization. In the training loop above, backpropagation is performed using the loss.backward() method, which calculates the gradients of the loss function with respect to the model's parameters. The optimizer.zero_grad() method is necessary to reset the gradients of the model's parameters to zero before performing backpropagation. This is because the gradients are accumulated by default, and we want to calculate the gradients for each batch separately, so we need to reset the gradients to zero before each batch.

In [8]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print('Accuracy on test data: {:.2f}%'.format(test_accuracy))

Accuracy on test data: 97.35%
